論文<br>
https://arxiv.org/abs/2105.06830<br>
<br>
GitHub<br>
https://github.com/msxie92/MangaRestoration<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/MangaRestoration_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GitHubからソースコードを取得

In [ ]:
%cd /content
!git clone https://github.com/msxie92/MangaRestoration.git

## ライブラリのインストール

In [ ]:
%cd /content/MangaRestoration

!pip install --upgrade gdown

## ライブラリをインポート

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2

# 学習済みモデルのセットアップ

In [ ]:
%cd /content/MangaRestoration
!mkdir -p release_model

if not os.path.exists('release_model/resattencv_manga_cons256.zip'):
  !gdown 'https://drive.google.com/uc?id=1sazt7jlvfR6KEjOp9Tq2GpjMe04uRgtn' \
    -O release_model/resattencv_manga_cons256.zip

!unzip release_model/resattencv_manga_cons256.zip -d release_model

# テスト画像のセットアップ

In [ ]:
%cd /content/MangaRestoration

!mkdir -p datazip/manga1/test flist/manga1

!wget -c https://github.com/msxie92/MangaRestoration/raw/main/examples/Akuhamu_020.jpg \
      -O datazip/manga1/test/test_01.jpg
!wget -c https://i.pinimg.com/736x/03/95/6b/03956bff323d9200a7e4d3b3020c5b58.jpg \
      -O datazip/manga1/test/test_02.jpg

## 画像縮小

In [ ]:
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

img = imread('datazip/manga1/test/test_02.jpg')
dst = cv2.resize(img, dsize=None, fx=0.5, fy=0.5)
cv2.imwrite('datazip/manga1/test/test_02.jpg', dst)

## flist作成

In [ ]:
!python scripts/flist.py \
  --path datazip/manga1/test \
  --output flist/manga1/test.flist

# Manga Restoration

In [ ]:
!python testreal.py -c configs/manga.json -n resattencv -s 256

In [ ]:
in1 = 'datazip/manga1/test/test_01.jpg'
out1 = 'release_model/resattencv_manga_cons256/results_real_00400/test_01_pred.png'
in2 = 'datazip/manga1/test/test_02.jpg'
out2 = 'release_model/resattencv_manga_cons256/results_real_00400/test_02_pred.png'

fig = plt.figure(figsize=(20, 20))

ax1 = fig.add_subplot(2, 2, 1) 
plt.title('test01 before', fontsize=16)
ax1.axis('off')

ax2 = fig.add_subplot(2, 2, 2)
plt.title('test01 after', fontsize=16)
ax2.axis('off')

ax3 = fig.add_subplot(2, 2, 3)
plt.title('test02 before', fontsize=16)
ax3.axis('off')

ax4 = fig.add_subplot(2, 2, 4)
plt.title('test02 after', fontsize=16)
ax4.axis('off')

ax1.imshow(imread(in1))
ax2.imshow(imread(out1))
ax3.imshow(imread(in2))
ax4.imshow(imread(out2))

# SRCNNと比較

In [ ]:
%cd /content

!git clone https://github.com/Mirwaisse/SRCNN.git
!curl https://raw.githubusercontent.com/chigozienri/SRCNN/master/models/model_4x.pth -o model_4x.pth

In [ ]:
%cd /content/SRCNN
!cp /content/MangaRestoration/datazip/manga1/test/test_01.jpg test_01.jpg

!python run.py \
  --zoom_factor 4 \
  --model /content/model_4x.pth \
  --image test_01.jpg \
  --cuda

In [ ]:
in1 = '/content/MangaRestoration/datazip/manga1/test/test_01.jpg'
out1 = '/content/MangaRestoration/release_model/resattencv_manga_cons256/results_real_00400/test_01_pred.png'
out2 = '/content/SRCNN/zoomed_test_01.jpg'

fig = plt.figure(figsize=(30, 10))

ax1 = fig.add_subplot(1, 3, 1) 
plt.title('test01', fontsize=16)
ax1.axis('off')

ax2 = fig.add_subplot(1, 3, 2)
plt.title('MangaRestoration', fontsize=16)
ax2.axis('off')

ax3 = fig.add_subplot(1, 3, 3)
plt.title('SRCNN', fontsize=16)
ax3.axis('off')

ax1.imshow(imread(in1))
ax2.imshow(imread(out1))
ax3.imshow(imread(out2))